In [1]:
import pandas as pd
from gensim.models import LdaModel
from gensim import corpora
import pickle
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [2]:
data = pd.read_csv("data/cleaned_data1.csv")

/var/folders/5q/yq3hk8g1793ckqmn2n3xpr6c0000gn/T/ipykernel_3543/1751319474.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data/cleaned_data1.csv")


## Load topic model, corpus and dictionary

In [3]:
model = LdaModel.load("lda_models/model30")
# Load corpus and dictionary
with open('corpus_and_dict.pkl', 'rb') as f:
    corpus, dictionary = pickle.load(f)

## Visualize topics

In [56]:
from pprint import pprint
top_topics = model.top_topics(corpus)  
top_topics

[([(0.027463017, 'even'),
   (0.027229311, 'now'),
   (0.01635985, 'never'),
   (0.016113337, 'right'),
   (0.01591412, 'bad'),
   (0.015865594, 'friend'),
   (0.015367166, 'well'),
   (0.015049828, 'back'),
   (0.014644586, 'hard'),
   (0.014208863, 'best'),
   (0.0141276, 'first'),
   (0.013845572, 'always'),
   (0.012199505, 'year'),
   (0.010812537, 'start'),
   (0.010724042, 'everything'),
   (0.010645424, 'long'),
   (0.010290676, 'new'),
   (0.010275607, 'relationship'),
   (0.009427303, 'happy'),
   (0.009381374, 'need')],
  -1.7160375336726756),
 ([(0.026362639, 'help'),
   (0.020163393, 'problem'),
   (0.019529866, 'different'),
   (0.017696982, 'talk'),
   (0.016014067, 'advice'),
   (0.014905186, 'situation'),
   (0.014681321, 'kind'),
   (0.0136652, 'issue'),
   (0.013587761, 'mean'),
   (0.012493776, 'idea'),
   (0.012479591, 'wrong'),
   (0.011220171, 'reason'),
   (0.010958158, 'actually'),
   (0.010531881, 'ask'),
   (0.010511818, 'less'),
   (0.009988068, 'question'),

In [5]:
pyLDAvis.enable_notebook()

# feed the LDA model into the pyLDAvis instance
lda_viz = gensimvis.prepare(model, corpus, dictionary)

In [6]:
pyLDAvis.display(lda_viz)

## Documents exploration

check again documents to see if they relate to other topics... and argue better for removing the general overlapping topics. Maybe remove documents without a decent prob of a topic? Argue they could be too vague for an analysis off comparison

In [7]:
data["topic_distribution"] = [model.get_document_topics(bow) for bow in corpus]
data["dominant_topic"] = [max(model.get_document_topics(bow), key=lambda x: x[1])[0] for bow in corpus]
data["prob_topic"] = [max(model.get_document_topics(bow), key=lambda x: x[1])[1] for bow in corpus]

In [55]:
with pd.option_context('display.max_colwidth', None):
    display(data[["topic_distribution", "dominant_topic"]])

,topic_distribution,dominant_topic
0,"[(1, 0.091077484), (2, 0.19981037), (8, 0.11064708), (9, 0.05697929), (10, 0.43843532)]",10
1,"[(0, 0.07607941), (1, 0.2018849), (8, 0.33099636), (10, 0.2948358)]",8
2,"[(0, 0.07237623), (1, 0.2134489), (8, 0.28447196), (10, 0.29890355)]",10
3,"[(0, 0.05791964), (1, 0.1347501), (7, 0.05429925), (8, 0.10180831), (9, 0.10339188), (10, 0.40576634)]",10
4,"[(1, 0.0826057), (7, 0.079685844), (8, 0.082812324), (9, 0.08609555), (10, 0.50374186)]",10
...,...,...
507511,"[(1, 0.2086316), (7, 0.2093879), (8, 0.072607756), (10, 0.41066173)]",10
507512,"[(1, 0.31717718), (8, 0.1416481), (10, 0.35113555)]",10
507513,"[(1, 0.21029706), (8, 0.1237867), (10, 0.521593)]",10
507514,"[(0, 0.068614565), (1, 0.088787764), (6, 0.15167311), (8, 0.17805305), (10, 0.33022785), (11, 0.1096014)]",10


In [11]:
data.dominant_topic.value_counts()

dominant_topic
10    405508
8      47095
1      37502
2       5866
7       3219
9       2420
0       2123
6       1714
4        818
5        597
3        497
11       157
Name: count, dtype: int64

### Topic 0 (LDAvis 1)

Posts with the highest probability have prob around 0.69 the top one and ~0.50 in the rest... they all talk about goal setting, more specifically the time dimension around it (short term vs long term)

Posts with lowest probability are some very vague and others still show some of this time dimension

IMPORTANT - Half of topic 5/6 (goal setting) also have a time dimension so perhaps topic 6 should be prioritized instead of this one (e.g. remove topic 0 and pass the documents to the other topics)

In [32]:
#topic0 = data[(data.dominant_topic == 0) & (data.prob_topic > 0.1)].sort_values("prob_topic", ascending=True)
topic0 = data[(data.dominant_topic == 0)].sort_values("prob_topic", ascending=False)
print(topic0["type"].value_counts())
with pd.option_context('display.max_colwidth', None):
    display(topic0[["type","cleaned_text", "dominant_topic", "prob_topic"]][0:20])

type
comment       1756
submission     367
Name: count, dtype: int64


,type,cleaned_text,dominant_topic,prob_topic
105247,comment,let me give a quantitatively accurate ratio of what you should do read read read read read read read read read read read read read read read read read read read read read read read read read read read read read read read read read read read read read read read read read read read read read read read read read read read read read read read read write write write write write write the problem with most writers imo is that they dont read enough all the shitty writing decisions they make are things other writers do better that they could learn from them if they only deflated their ego and read more,0,0.696981
141570,comment,there are long term goals and short term goals to get to the long term one you need to go through the short term ones personally i suck at long term goal but i do well focusing on the short term ones in time the short term ones will make that long term goal just look like the next step also achievement is an exhilarating feeling setting short term goals and achieving them will make you feel more productive and itll also give u results so that you can prove to urself that ur not just wasting time,0,0.625006
142053,comment,setting goal is good but you have to be crystal clear how you will achieve your goal a good way is to use the smart framework to set your goals one reason why most people do not achieve the goals they set at the start of the year is they lose sight of their goals i suggest you set short term medium and long term goals ie daily goals that will help you attain your long term goals have weekly yearly goals and 5 yearly goals feel free to revisit these goals as your circumstances will change and some of the goals will be irrelevant,0,0.572464
494613,comment,long term goals can drive short term goals if youre looking to get a new home then short term goals might be researching where you want to live how much money youll need then starting a savings plan thinking about furniture etc if your long term goal is to run a marathon you might have a short term goal of running 20 minutes a day and working your way up to a 5k race then 10k race etc the long term and short term goals can definitely relate to each other,0,0.546482
199338,comment,keep it simple plan in 3s start with the big picture maybe life goals or 10 year goals then in a lesser time what 3 goals would put you closer to those now you may be down to 1 year goals 3 goals to get to the next level then 3 6month goals to get you to your 1 year goals then 3 1month goals to get to your 6month or 3month then 3 goals this week to get you to your 1 month goal then 3 goals today to get to your 1 week goal it may seem like a lot at first but when you think of the big picture its not and when you do it the 3 goals today shouldnt seem too big 3 simple goals at a time that get you closer to your big goals tomorrow 3 more the next day 3 more say you do it 5 days a week 50 weeks a year taking in time off and weekends that is 750 goals accomplished in 1 year now that is progress,0,0.542681
4790,submission,hi guys id like to get into the routine of setting long term and short term goals i know every successful person claims goalsetting is absolutely crucial to being successful instead of every morning writing on my yellow pad id rather see if theres an app that i can set both short term and long term goals short term id like to set new goals to complete by the end of the day and to review my long term goals for instance long term goal 1 100000 of passive income per year by age 30 short term goal for today 1 make 100 phone calls add 5 people to my database completed check it off and see my progress over the course of the week month etc,0,0.540592
302189,comment,start with breaking big things you need to achieve into small and reachable goals and once you are on a roll of setting and achieving the small ones gradually make them bigger eventually you will be achieving bigger things and that should help ex

### Topic 1 (LDAvis 2)

Words seem like problem solving or smth.
Documents are around mindset, growth, self awareness, beliefs etc. Make sense as it overlaps with topic 2/3

In [31]:
#topic1 = data[(data.dominant_topic == 1) & (data.prob_topic > 0.1)].sort_values("prob_topic", ascending=True)
topic1 = data[(data.dominant_topic == 1)].sort_values("prob_topic", ascending=False)
print(topic1["type"].value_counts())
with pd.option_context('display.max_colwidth', None):
    display(topic1[["type","cleaned_text", "dominant_topic", "prob_topic"]][0:20])

type
comment       31557
submission     5945
Name: count, dtype: int64


,type,cleaned_text,dominant_topic,prob_topic
95990,submission,while reading a blog these key points stuck out to me significance of forgiveness in relationships forgiveness is essential in relationships as it allows individuals to move past hurts and resentments fostering emotional healing and growth impact of kindness on emotional state acts of kindness positively affect ones emotional state bringing joy gratitude and fulfillment and enhancing overall happiness and wellbeing transformative nature of forgiveness forgiveness is transformative altering emotions attitudes and behaviors leading to healing growth and peace interconnection of love compassion and forgiveness love and compassion are crucial for fostering forgiveness love nurtures the capacity to forgive while compassion promotes understanding and empathy cultivating kindness and forgiveness cultivating these qualities involves intentional acts and mindset shifts such as practicing empathy gratitude and generosity practical steps towards forgiveness and kindness reflection acknowledging ones own role in conflicts and choosing to let go are practical steps towards cultivating forgiveness and kindness forgiveness as an act of selfcompassion forgiving is an act of selfcompassion that helps release negative emotions and facilitates healing and growth benefits of forgiveness to others forgiveness can help others by providing opportunities for reflection growth and reconciliation loves role in forgiveness love is central to forgiveness promoting patience empathy and understanding enabling individuals to extend grace and compassion forgiveness without apology forgiveness can occur independently of an apology from the other person as it is a personal journey towards peace forgiveness as a spiritual practice forgiveness aligns with many spiritual traditions as a pathway to inner peace and harmony forgiveness contributing to personal growth it enables moving beyond past hurts and breaking the cycle of resentment fostering selfawareness and empowerment transformative power of kindness and forgiveness these acts can shift narratives from hurt to healing and growth highlighting our capacity for forgiveness and the freedom it brings cultivating forgiveness feelings it starts with accepting the truth of a situation and shifting thoughts from blame and anger to understanding and compassion importance of letting go of grudges letting go of grudges is vital for mental and emotional health as it allows for peace and positive progression i believe its important to have these qualities because it strengthens bonds and allows the individual to gain an understanding of empathy and allows them to grow as a person allowing them a strong healthy and fullfilling relationship comment your thoughts below,1,0.791692
80650,submission,selfawareness is a fundamental aspect of personal development and overall wellbeing it involves consciously understanding oneself including strengths weaknesses emotions and motivations in this article we will explore the importance of selfawareness and how cultivating this skill can lead to personal growth improved relationships and a more fulfilling life understanding yourself selfawareness allows us to gain a deeper understanding of who we are at our core it involves being aware of our thoughts emotions and behaviors and how they influence our actions and interactions with others by understanding our values beliefs and priorities we can make more intentional choices aligned with our authentic selves this selfknowledge forms the foundation for personal growth and enables us to live a more purposeful life embracing strengths and weaknesses selfawareness helps us identify our strengths and weaknesses when we know our strengths we can leverage them to excel in our endeavors and find fulfillment in areas where we naturally excel recognizing our weaknesses allows us to seek opportunities for growth and improvement selfawareness enables us to acknowledge our limitations and

### Topic 2 (LDAvis 3)

Seems to be about changes, decisions/choices, mindset etc. Also a lot of habits/routine stuff

In [36]:
topic2 = data[(data.dominant_topic == 2) & (data.prob_topic > 0.0)].sort_values("prob_topic", ascending=True)
#topic2 = data[(data.dominant_topic == 2)].sort_values("prob_topic", ascending=False)
print(topic2["type"].value_counts())
with pd.option_context('display.max_colwidth', None):
    #display(topic2[["type","cleaned_text", "dominant_topic", "prob_topic"]][0:20])
    display(topic2[0:20])

type
comment       4724
submission    1142
Name: count, dtype: int64


,Unnamed: 0,id,created,author,score,num_comments,link,cleaned_text,word_count,type,link_id,year,month,topic_distribution,dominant_topic,prob_topic
154651,154651,e6k6h15,2018-09-24 12:39,u/SpartEng76,1,NaN,https://www.reddit.com/r/selfimprovement/comments/9ibu9v/what_do_you_guys_do_in_your_freetime/e6k6h15/,chores mostly on the weekends i cut the grass do some maintenance in the yard or around the house clean do laundry walk the dogs buy food for the week exercise wash my car run errands do house projects etc i usually have a mental list of things i want to accomplish and once i get them done i feel good about having a beer and just relaxing during the week its mostly work work out cook dinner eat watch some tv,82,comment,t3_9ibu9v,2018,9,"[(0, 0.11122298), (1, 0.06790584), (2, 0.15024339), (3, 0.07020933), (7, 0.050131567), (8, 0.09440754), (9, 0.14810139), (10, 0.1439927), (11, 0.10663296)]",2,0.150238
337521,337521,ibfzltd,2022-06-06 21:42,u/Blindman_in_the_cave,2,NaN,https://www.reddit.com/r/selfimprovement/comments/v6ipnz/dealing_with_binge_consumption_and_compulsive/ibfzltd/,okay yup i see you the junk food thing is a global conglomerate manipulating your brain chemistry for their profit to your detriment if the masturbation includes porn consumption thats akin to the junk food manipulation as are the net and the video games all of these are industries that are using your dopamine response to enrich themselves and their stockholders at your expense you need to be the one in charge of your dopamine you need to be the one to set goals and accomplish goals and set higher goals keep it tight and attainable and reward yourself along the way out of bed before 7 bed made showered shaved and coffee in hand by 730 5 minutes of scrolling gone wild hair combed teeth brushed and in route by 8 another 5 of gone wild continue this throughout your day set achievable goals and reward your successes,149,comment,t3_v6ipnz,2022,6,"[(0, 0.14940387), (1, 0.07691699), (2, 0.15551092), (3, 0.07384258), (6, 0.077957824), (7, 0.062352132), (8, 0.10998233), (9, 0.15032935), (10, 0.12650867)]",2,0.155511
501376,501376,kcozgin,2023-12-09 16:35,u/eljackson,1,NaN,https://www.reddit.com/r/selfimprovement/comments/18ejqru/is_it_brain_fog_or_i_am_just_dumb/kcozgin/,you mention traits that align with common adhdasd pairings brain foginattentivenessdisorganisation adhd as well as social anxiety friendship difficulties motor coordination problems asd when unmanaged downstream symptoms aligning with anxiety depression ocdlike traits and mental disorientation can manifest start with conducting an adhdasd screening alongside for other neurodivergent traits with a physicianpsychiatrist like other folks mentioned too get your blood work checked alongside maybe even sleep apnea tests food intolerancesallergies or other screenings for chronic fatigue if a doctor recommends it heres more general questions to flesh out what youre going through lifestyle changes may alleviate the intensity of some of your symptoms how many hours a day are you on social mediavideo gamesother dopamine addictive activities whats your regular hours of sleep a night do you wake up feeling refreshed or tired how many hours and what type of exercise do you do weekly what does your diet look like how does your hydration look is brain fog something that is present in other members of your family is the brain fog at a constant intensity throughout the day is it only there for certain scenarios and activities how do you feel around big crowds with lots of noises and stimuli how do you feel when having to do something outside of your social comfort zone were you considered inattentive or hyperactive as a child did you have friendships in early childhood or did you play alone who at all do you currently have social interactions with family online friends do you want social connections or do you feel indifferentcomfortable alone,262,comment,t3_18ejqru,2023,12,"[(1, 0.101063326), (2, 0.165693), (4,

### Topic 3 (LDAvis 4)

Routine

In [39]:
topic3 = data[(data.dominant_topic == 3) & (data.prob_topic > 0.0)].sort_values("prob_topic", ascending=False)
#topic3 = data[(data.dominant_topic == 3)].sort_values("prob_topic", ascending=False)
print(topic3["type"].value_counts())
with pd.option_context('display.max_colwidth', None):
    display(topic3[["type","cleaned_text", "dominant_topic", "prob_topic"]][0:20])


type
comment       410
submission     87
Name: count, dtype: int64


,type,cleaned_text,dominant_topic,prob_topic
367697,comment,i wouldnt say the post is anti video games liking video games is fine i personally love video games the issue is when you spend all of your free time playing video games watching netflix and dont have any other hobbies especially more active hobbies like i said i love video games but i also hit the gym train muay thai go hiking etc its just about being well rounded,3,0.493976
350202,comment,honestly giving up video games for me helped me a lot since video games would always be so addicting i loved videos games for such a long time but i realized i had so many other opportunities that i was losing because i was playing video games i gave up video games at around 17,3,0.483632
371047,comment,you set up a false dichotomy that the opposite of playing video games is being 100 productive all the time there are fun things that also lead to much more fruitful outcomes than video games like talking with a friend which deepens your relationship or reading about a topic youre fascinated in which gets you more educated should video games be avoided not necessarily but there are activities just as fun as video games that add more to your future than video games do so video games should be played in moderation to allow time for those activities that do add more,3,0.469969
131202,comment,ok so you want to find your passion you think some types of games are your passion but you dont know if its the game or something that youre doing inside the game that makes you like it doing things in real life is not the same as in video games pick the top 5 or so of your favorite games and note what makes each interesting to you game one a interesting thing one b interesting thing two then go out and find realworld jobs or people who are doing those interesting things that you see in video games learn about them and do them yourself in real life to see if you actually like them if it turns out you actually really like one of those things lay off the video games and focus on it if you go and dont like any of them in real life then you have to decide is spending time playing video games beneficial or helpful if you really enjoy vg and think you can make a career out of playing video games then you have to go out and do research on how exactly to do this again lay off the games and do research on how you can make money doing it my general views on video games are that they waste time and money depending on how much you play im assuming youre younger and you still got a lot of time ahead of you so be honest with yourself and set your priorities to match your goals in life hope this helped,3,0.455638
204939,comment,my only advice would be to work on some hobbies youre interested do you have any hobbies do you create any kind of art writing or painting or playing music what are your interests what do you enjoy learning about if your hobbies are things like playing video games and watching movies maybe consider picking up an additional hobby that lets you be more creative rather than consumptive just do things that you find interesting if other people think that is boring then thats their problem youll find someone eventually with similar interests,3,0.434425
144161,comment,this may seem over the top but try quitting video games altogether the truth is video games will always be more stimulating then reading a book notice how i said stimulating instead of fun so your body will always crave video games more then books best way to choose books over video games is to quit video games all together treat video games like how you would any other addiction avoid anything that reminds you of it realize that the cravings go away over time and most importantly fill the void with a new hobby,3,0.431008
454395,comment,this is an incredible statement may i ask how old you are i myself was around in the early 1980s and playing video games and also do you mean youve wasted your video gaming life meaning the part of yo

### Topic 4 (LDAvis 5)

Topic words seem coherent but the categorization here doesn't make sense. It's getting things of like social media

In [41]:
#topic4 = data[(data.dominant_topic == 4) & (data.prob_topic > 0.20)].sort_values("prob_topic", ascending=False)
topic4 = data[(data.dominant_topic == 4)].sort_values("prob_topic", ascending=True)
print(topic4["type"].value_counts())
with pd.option_context('display.max_colwidth', None):
    display(topic4[["type","cleaned_text", "dominant_topic", "prob_topic"]][0:20])

type
comment       698
submission    120
Name: count, dtype: int64


,type,cleaned_text,dominant_topic,prob_topic
237768,comment,i loved living on my own and you can do all kinds of things to pass the time go out hiking or sightseeing in your area read booksjoin a virtual book club play video gamesjoin some gaming groups do group sports call up friends or family and catch up learn some skills like coding a new language how to create art etc do some house projects declutter renovate your space think of new stuff to do with your kids when you see them next time organize photosfiles on your computer try cooking yoga meditation volunteer for a cause of your choice mentor younger people in your field look into virtual classescertifications watch some good movies or shows if youre nervous breaking the ice see if there is a local toastmasters in your area a group that helps people improve their public speaking and social skills its also great to keep a daily journal so you can catalogue your achievements and look back to see all the goals you accomplished,4,0.154062
369612,comment,board game clubs quiet structured environment i attend a chess club but there are many different kinds check your local library they probably have a list of groups that meet there like my local library has a sewing club a book club chess club and a scrabble club also classes especially hobby classes like cookingbaking pottery basket weaving ect ive had a lot of success meeting people this way and i learn a lot i know you dont want sports but what about yoga dance or taichi classes i find they helped my mental health and a great way to get out and about i hope it gets better for you,4,0.172612
134308,comment,besides working on your self esteem and sense of intrinsic self worth for which i suspect cognitive behavioral therapy or other therapy techniques would do wonders health insurance plans cover this with small copays now for in network social workers and therapists i think you should focus on improving your social skills in general and forcing yourself to have in person interactions with others on a more regular basis join clubs meetup for any hobbies go to the public library for free community activities anything to get you out of the house and learning how to read social cues so when you do get to school you are prepared to speak to women with respect and skill and you have real world experiences to tell stories about,4,0.176736
375383,comment,also checkout groups that go hiking etc search on the internet for local groups depending on your winters it can be tougher but options are available if physical activities are not your thing look into local choir or crafting groups cities libraries etc are offering more gathering activities card games gaming etc,4,0.180832
34774,submission,porn addiction does masturbating to pornography amp playing video games give you social anxiety in this video johnny berba answers the question does masturbating to pornography amp playing video games give you social anxiety self improvement really comes down to facing your fears and insecurities on a practical level which i do explain in this video which ties into overcoming porn addiction and social anxiety for me self improvement is also about treating other people with kindness and being the best person you can be on a daily basis taking small action steps every single day is one of the quickest ways to build confidence and selfesteem ampx200b jb,4,0.181291
38718,submission,i want to make friends since i wasted high school and college was not ideal for me socially i need new ways to make friends and create a social circle what are some groups i dont have interests in many things i am shamefully a constant consumer of youtube and a pc gamer who plays one game my few actual interests are not very social ones cars foraging fossil hunting video editing astrology dream interpretation music im bad at it comedy is my one social interest and others in comedy are usually fickle how can i develop more natural interests it has always been slow for me t

### Topic 5 (LDAvis 6)

Mental health? Why not categorizing correctly?

In [43]:
#topic5 = data[(data.dominant_topic == 5) & (data.prob_topic > 0.20)].sort_values("prob_topic", ascending=False)
topic5 = data[(data.dominant_topic == 5)].sort_values("prob_topic", ascending=False)
print(topic5["type"].value_counts())
with pd.option_context('display.max_colwidth', None):
    display(topic5[["type","cleaned_text", "dominant_topic", "prob_topic"]][0:20])

type
comment       537
submission     60
Name: count, dtype: int64


,type,cleaned_text,dominant_topic,prob_topic
319115,comment,therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy therapy,5,0.816276
388133,comment,i think theres some benefit to differentiating the two so that you can recognize causation vs correlation but i agree with the sentiment especially in mental healthcare providers might be too fast to dismiss physical health issues because well you have depression no reason to actually listen to you and refer you for tests fatigue pain nausea depression seeking out help for chronic physical symptoms is so difficult if you have a mental illness on file providers should really start seeing humans as a full package improving your mental health or recovering from trauma is near impossible if your life is dominated by physical ailments that hinder your ability to function and untreated physical issues can cause depression anxiety suicidal ideation etcand if that is the case medicating the symptoms wont fix the core issue only medical care meant to address the underlying physical condition will,5,0.491624
81880,submission,hey fellow students as we navigate our academic journey we often confront stress anxiety and other mental health challenges a group of students has embarked on a project to improve mental health resources and wed really appreciate your thoughts regardless of whether you regularly see a therapist have tried numerous apps never used any resources or dont typically engage with mental health care your experience is valuable to us lets have an open conversation about our mental health journeys x200b 1 access to professional help how easy or difficult has it been for you to access professional help for your mental health if youve sought it out what was your experience like if you havent what has stopped you 2 mental health apps if youve used any mental health apps have they been helpful what did you like or dislike about them 3 improvements whats missing in the current landscape of mental health resources that could make your mental health journey better 4 selfmanagement how do you manage your mental health do you have any selfcare routines or strategies that work well for you 5 learning about mental health where do you usually learn about mental health and its management do you have any goto resources or favorite platforms were here to listen learn and hopefully help create something that truly resonates with the needs of students lets shed light on mental health and work towards better support for all of us,5,0.490402
427299,comment,to add a better explanation i found this definition psychiatrists are medical doctors who are able to prescribe medications which they do in conjunction with providing psychotherapy though medical and pharmacological interventions are often their focus though many psychologists hold doctorate degrees they are not medical doctors and most cannot prescribe medications rather they solely provide psychotherapy which may involve cognitive and behavioral interventions and here also about therapists therapists require master degrees and approval of their licensing boards to practice in the mental health field therapists provide mental health diagnosis and develop a treatment plan therapists work in offices hospitals treatment centers and group homes there are many different types of therapy such as play therapy cognitive behavioral therapy animalassisted therapy dialectal behavioral therapy and many others therapists cannot recommend or order medications but they can refer you for evaluation for medication or other treatments so for diagnosis medication and mental physical analysis psychiat

### Topic 6 (LDAvis 7)

Dating and sex

In [45]:
#topic6 = data[(data.dominant_topic == 6) & (data.prob_topic > 0.20)].sort_values("prob_topic", ascending=False)
topic6 = data[(data.dominant_topic == 6)].sort_values("prob_topic", ascending=True)
print(topic6["type"].value_counts())
with pd.option_context('display.max_colwidth', None):
    display(topic6[["type","cleaned_text", "dominant_topic", "prob_topic"]][0:20])

type
comment       1646
submission      68
Name: count, dtype: int64


,type,cleaned_text,dominant_topic,prob_topic
236010,comment,1 physical selfcare good hygiene reasonable tidiness in appearance physical health you dont have to douse yourself in cologne dress yourself like a model and have the body of a gymnast but brushed teeth and hair clean jeans and unwrinkled shirt no coughing healthy skin etc 2 have something you find joy in a good hobby or interest lets you meet a variety of people makes you interesting to talk to keeps your mind agile and increases your selfconfidence,6,0.172206
448842,comment,honey god he is a living god he hears all prayers you have to build up your confidence it doesnt require money i want you to date yourself i know that sounds strange i want you plan this date be creative write what kind activity you lwant to do on your date what will you wear casual or a suit what location you go what food you can bring a meal from home bring no one with you why not because iits your dateyour the only that matters each date you plan for yourself will build your confidence up up up i will put a good word for humliation i read your post,6,0.175517
406678,comment,the problem with questions like these is that we cant quantify the degree of ugly and social anxiety you are talking about there are also many people who believe they are uglywhen they are just average or even good looking because of past experiences the truth is the only people who are actually doomed are ugly short guys with social anxiety most guys dont fall into that category though and have relatively equal opportunity as others,6,0.188044
362320,comment,thanks for pointing this out im a therapist and i just wish that folks would stop saying things like porn addiction that is not something we can diagnosis someone with generally there is something else going on underneath the behavior id also like to say that a ton of these no porn no masturbation things are perpetrated by religious organizations they say things like porn changes your brain you cant engage in normal sex bla bla bla and its all nonsense porn and masturbation are not things that people need to be shamed into quitting lack of sex education understanding of human sexuality media literacy porn is not real life and a host of other things have created this thing where we think that you are immoral or incapable of a normal sex life if you watch porn anyways im happy to send anyone information peer reviewed psych journals etc on this topic and yes im a sex positive therapist but my main area of work is with people that are victimsurvivors of rape and interpersonalfamily violence i also work with lots of religious trauma and no porn does not cause rape much like video games do not cause school shootings,6,0.188402
421516,comment,to be fair the male body and brain suffer the most from porn and masturbation addiction men get sudden dicrease in energy motivation become lethargic and complacent irritable due to lack of testosterone consequently it reduces the desire to pursue real women i believe due to nature of our bodies men benefit a lot more from disciplining ourselves into abstaining from these activities,6,0.190974
463281,comment,i dont really think i cant find any woman who wont sleep with me as in i dont think i cant get married i certainly can but i dont want to i dont want to sleep with a woman and i really dont want to use a sex worker its against my religion to have sex outside of marriage anyways and its not worth the stds pregnancy feelings of emptiness,6,0.194104
64652,submission,to put it simply optimise your male hormones this is something that cannot be overlooked by any man undergoing self improvement hormones have a huge impact on your physical appearance as well as your mood amp general psychology as a person think alpha male in appearance and mindset get the following bloodwork done as a start testosterone free testosterone dht shbg prolactin estradiol cortisol lhfsh tsh free t3 t4 the goal here is to optimise your androgens free t dht meanwhi

### Topic 7 (LDAvis 8)

Docs with low probability aren´t good
Job and school posts

In [51]:
#topic7 = data[(data.dominant_topic == 7) & (data.prob_topic > 0.20)].sort_values("prob_topic", ascending=False)
topic7 = data[(data.dominant_topic == 7)].sort_values("prob_topic", ascending=False)
print(topic7["type"].value_counts())
with pd.option_context('display.max_colwidth', None):
    display(topic7[["id","type","cleaned_text", "dominant_topic", "prob_topic"]][0:20])

type
comment       2472
submission     747
Name: count, dtype: int64


id        type  \
396929  j3umkfb     comment   
2929     3a12at  submission   
380221  iyjviop     comment   
497579  kaqf0y5     comment   
208755  fmdwaw2     comment   
410310  j9hw4rj     comment   
84606   15m4y3z  submission   
291451  hh648ds     comment   
377700  ixoni84     comment   
17263    b9h5oq  submission   
54836    tcpqjj  submission   
218767  fwamdlr     comment   
65038    y7rgw4  submission   
261732  gyijm5i     comment   
326041  i42bgn0     comment   
99416   ccset28     comment   
126410  dftejld     comment   
144827  dz1thcw     comment   
159915  eaoude2     comment   
335503  ia6j1yn     comment   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

### Topic 8 (LDAvis 9)

In [77]:
#topic8 = data[(data.dominant_topic == 8) & (data.prob_topic > 0.20)].sort_values("prob_topic", ascending=False)
topic8 = data[(data.dominant_topic == 7)].sort_values("prob_topic", ascending=True)
print(topic8["type"].value_counts())
with pd.option_context('display.max_colwidth', None):
    display(topic8[["id","type","cleaned_text", "dominant_topic", "prob_topic"]][0:20])

type
comment       2472
submission     747
Name: count, dtype: int64


id        type  \
315104  hvpgk62     comment   
435744  jlb530g     comment   
314450  hv9sd6z     comment   
312642  htzchez     comment   
162980  ecv8k1v     comment   
419219  jd47r5u     comment   
503432  kdrpa9y     comment   
505274  kequg8t     comment   
14944    a8ktzx  submission   
42139    n85hv5  submission   
441700  jo40szz     comment   
478805  k2r10gb     comment   
438938  jn1myj9     comment   
118528  d5pos72     comment   
164415  edx5fqi     comment   
29533    gr45xp  submission   
500715  kcbg2li     comment   
322766  i1lxodz     comment   
94358   17zrkol  submission   
375415  iwqdgig     comment   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

### TRY ALL

In [80]:
#topic8 = data[(data.dominant_topic == 8) & (data.prob_topic > 0.20)].sort_values("prob_topic", ascending=False)
topic = data[(data.dominant_topic == 7)].sort_values("prob_topic", ascending=False)
print(topic["type"].value_counts())
with pd.option_context('display.max_colwidth', None):
    display(topic[["id", "cleaned_text", "dominant_topic", "prob_topic", 
                   "topic_distribution"]][0:20])

type
comment       2472
submission     747
Name: count, dtype: int64


id  \
396929  j3umkfb   
2929     3a12at   
380221  iyjviop   
497579  kaqf0y5   
208755  fmdwaw2   
410310  j9hw4rj   
84606   15m4y3z   
291451  hh648ds   
377700  ixoni84   
17263    b9h5oq   
54836    tcpqjj   
218767  fwamdlr   
65038    y7rgw4   
261732  gyijm5i   
326041  i42bgn0   
99416   ccset28   
126410  dftejld   
144827  dz1thcw   
159915  eaoude2   
335503  ia6j1yn   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             